# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc29c9dff70>
├── 'a' --> tensor([[ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440]])
└── 'x' --> <FastTreeValue 0x7fc29c9df970>
    └── 'c' --> tensor([[-1.9376, -0.3890, -0.9870, -1.7412],
                        [ 0.0291, -0.1666, -0.4339,  0.1297],
                        [ 0.6328,  0.9968,  1.5475, -0.6239]])

In [4]:
t.a

tensor([[ 0.3768,  0.8949,  1.0881],
        [ 1.7863,  0.0154, -0.3440]])

In [5]:
%timeit t.a

64 ns ± 0.0616 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc29c9dff70>
├── 'a' --> tensor([[-0.3512,  1.5459,  0.1499],
│                   [-1.0810, -0.8687,  0.3485]])
└── 'x' --> <FastTreeValue 0x7fc29c9df970>
    └── 'c' --> tensor([[-1.9376, -0.3890, -0.9870, -1.7412],
                        [ 0.0291, -0.1666, -0.4339,  0.1297],
                        [ 0.6328,  0.9968,  1.5475, -0.6239]])

In [7]:
%timeit t.a = new_value

67.5 ns ± 0.0958 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440]]),
    x: Batch(
           c: tensor([[-1.9376, -0.3890, -0.9870, -1.7412],
                      [ 0.0291, -0.1666, -0.4339,  0.1297],
                      [ 0.6328,  0.9968,  1.5475, -0.6239]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3768,  0.8949,  1.0881],
        [ 1.7863,  0.0154, -0.3440]])

In [11]:
%timeit b.a

59.2 ns ± 0.0461 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0415, -0.6360, -0.2242],
               [-1.0430, -0.7022,  0.2348]]),
    x: Batch(
           c: tensor([[-1.9376, -0.3890, -0.9870, -1.7412],
                      [ 0.0291, -0.1666, -0.4339,  0.1297],
                      [ 0.6328,  0.9968,  1.5475, -0.6239]]),
       ),
)

In [13]:
%timeit b.a = new_value

503 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

819 ns ± 2.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 52.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 628 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc27415c2b0>
├── 'a' --> tensor([[[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]],
│           
│                   [[ 0.3768,  0.8949,  1.0881],
│                    [ 1.7863,  0.0154, -0.3440]]])
└── 'x' --> <FastTreeValue 0x7fc274170a00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 63.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc274170b20>
├── 'a' --> tensor([[ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440],
│                   [ 0.3768,  0.8949,  1.0881],
│                   [ 1.7863,  0.0154, -0.3440]])
└── 'x' --> <FastTreeValue 0x7fc1f26ebf40>
    └── 'c' --> tensor([[-1.9376, -0.3890, -0.9870, -1.7412],
                        [ 0.0291, -0.1666, -0.4339,  0.1297],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 80.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.8 µs ± 83.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]],
       
               [[ 0.3768,  0.8949,  1.0881],
                [ 1.7863,  0.0154, -0.3440]]]),
    x: Batch(
           c: tensor([[[-1.9376, -0.3890, -0.9870, -1.7412],
                       [ 0.0291, -0.1666, -0.4339,  0.1297],
                       [ 0.6328,  0.9968,  1.5475, -0.6239]],
         

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 163 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440],
               [ 0.3768,  0.8949,  1.0881],
               [ 1.7863,  0.0154, -0.3440]]),
    x: Batch(
           c: tensor([[-1.9376, -0.3890, -0.9870, -1.7412],
                      [ 0.0291, -0.1666, -0.4339,  0.1297],
                      [ 0.6328,  0.9968,  1.5475, -0.6239],
                      [-1.9376, -0.3890, -0.9870, -1.7412],
                      [ 0.0291, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 229 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
